In [6]:
!pip install pyspark[sql,ml,mllib]

In [7]:
!pip install -q findspark

In [8]:
# Import the libary
import findspark
# Initiate findspark
findspark.init()
# Check the location for Spark
findspark.find()

'C:\\Users\\suhon\\anaconda3\\envs\\pyspark\\lib\\site-packages\\pyspark'

In [9]:
import pyspark

from pyspark.sql import SparkSession

# Create a Spark Session

spark = SparkSession.builder.config("spark.driver.host", "localhost").\
        config("spark.driver.memory", "16g").getOrCreate()

# Create Dataframe of Desired Data Columns 

In [10]:
import pandas as pd

products=pd.read_csv("olist_products_dataset.csv")

# add an index for each product ID
products['product_id_index']=products.index

#drop unwanted columns in datasets
products = products.drop(["product_name_lenght","product_description_lenght","product_photos_qty","product_weight_g","product_length_cm","product_height_cm","product_width_cm"], axis = 1)
order_items = pd.read_csv("olist_order_items_dataset.csv")
order_items = order_items.drop(["order_item_id","seller_id","shipping_limit_date","price","freight_value"], axis = 1)
orders_dataset = pd.read_csv("olist_orders_dataset.csv")
orders_dataset = orders_dataset.drop(["order_status","order_purchase_timestamp","order_approved_at","order_delivered_carrier_date","order_delivered_customer_date","order_estimated_delivery_date"], axis = 1)

customers_dataset = pd.read_csv("olist_customers_dataset.csv")

customers_dataset = customers_dataset.drop(["customer_zip_code_prefix","customer_city","customer_state",], axis = 1)
orders_reviews = pd.read_csv("olist_order_reviews_dataset.csv")
orders_reviews = orders_reviews.drop(["review_id","review_comment_title","review_comment_message","review_creation_date","review_answer_timestamp"], axis = 1)

#merge different datacharts
ds = pd.merge(products, order_items, on = 'product_id', how ='inner')
ds = pd.merge(ds, orders_dataset, on = 'order_id', how ='inner')
ds = pd.merge(ds, customers_dataset, on = 'customer_id', how = 'inner')
ds = pd.merge(ds, orders_reviews, on = 'order_id', how = 'inner')

ds.head()

,product_id,product_category_name,product_id_index,order_id,customer_id,customer_unique_id,review_score
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,0,e17e4f88e31525f7deef66779844ddce,f8a3e963a310aa58b60a5b1fed5bceb5,b1a1199364a4a7fe27c4486ab63f550d,5
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,1,5236307716393b7114b53ee991f36956,03fc97548af8f58fefc768d12b546c9c,4b86049cb99e4aa774031daa9cd18f18,5
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,2,01f66e58769f84129811d43eefd187fb,e41819d1c95c12c9ce495b630eab8aee,f63805d9c7edb84d92413af34b86a39c,5
3,cef67bcfe19066a932b7673e239eb23d,bebes,3,143d00a4f2dde4e0364ee1821577adb3,322162b5ca010c2b059cb5224dd818b1,619e926d09b26efbd5180368b1ddc874,1
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,4,86cafb8794cb99a9b1b77fc8e48fbbbb,c11c31965ff02cc1d7132df8edfcbc22,ad353b4fb0e294adc4eda48af73e68a6,5


In [11]:
# create an unique index for customer unique id

customer_id_pd=ds['customer_unique_id'].value_counts().reset_index(name="count").rename(columns={'index':'customer_unique_id'})

customer_id_pd['customer_unique_id_index']=customer_id_pd.index

customer_id_pd1=customer_id_pd.drop(['count'], axis=1)

customer_id_pd1

,customer_unique_id,customer_unique_id_index
0,c8460e4251689ba205045f3ea17884a1,0
1,d97b3cfb22b0d6b25ac9ed4e9c2d481b,1
2,4546caea018ad8c692964e3382debd19,2
3,c402f431464c72e27330a67f7b94d4fb,3
4,0f5ac8d5c31de21d2f25e24be15bbffb,4
...,...,...
94716,2b7a6611fe5722fbb667e1758ccf772b,94716
94717,7fc2e54bf3d557c5085b650c943d0fef,94717
94718,d9972a3678b1997f77ce16a51ccc5cc0,94718
94719,96ce6d85f1fd6f113fa1298a27047f2a,94719


In [12]:
ds1 = pd.merge(ds, customer_id_pd1, on = 'customer_unique_id', how ='inner')

ds1

,product_id,product_category_name,product_id_index,order_id,customer_id,customer_unique_id,review_score,customer_unique_id_index
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,0,e17e4f88e31525f7deef66779844ddce,f8a3e963a310aa58b60a5b1fed5bceb5,b1a1199364a4a7fe27c4486ab63f550d,5,18226
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,1,5236307716393b7114b53ee991f36956,03fc97548af8f58fefc768d12b546c9c,4b86049cb99e4aa774031daa9cd18f18,5,82684
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,2,01f66e58769f84129811d43eefd187fb,e41819d1c95c12c9ce495b630eab8aee,f63805d9c7edb84d92413af34b86a39c,5,65693
3,cef67bcfe19066a932b7673e239eb23d,bebes,3,143d00a4f2dde4e0364ee1821577adb3,322162b5ca010c2b059cb5224dd818b1,619e926d09b26efbd5180368b1ddc874,1,65692
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,4,86cafb8794cb99a9b1b77fc8e48fbbbb,c11c31965ff02cc1d7132df8edfcbc22,ad353b4fb0e294adc4eda48af73e68a6,5,65691
...,...,...,...,...,...,...,...,...
112367,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,32949,59e88b7d3675e89aceaf86f372d3bc9a,e750210dde7e68dbd5194c0809e97f7e,4d1bbadcfd9bdc738e54599627870319,4,8942
112368,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,32949,6e4465d771f02e4fe335225de3c6c043,0f139d1b208a62a37671423e5eff5016,2cc893f1618b7317170ba2456cf76117,5,2991
112369,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,32949,6e4465d771f02e4fe335225de3c6c043,0f139d1b208a62a37671423e5eff5016,2cc893f1618b7317170ba2456cf76117,5,2991
112370,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,32949,6e4465d771f02e4fe335225de3c6c043,0f139d1b208a62a37671423e5eff5016,2cc893f1618b7317170ba2456cf76117,5,2991


In [13]:
ds1['review_score'].value_counts()

5    63525
4    21315
1    14235
3     9423
2     3874
Name: review_score, dtype: int64

In [14]:
ds1.isnull().sum()

product_id                     0
product_category_name       1598
product_id_index               0
order_id                       0
customer_id                    0
customer_unique_id             0
review_score                   0
customer_unique_id_index       0
dtype: int64

In [15]:
# drop row where product category name is null

ds2=ds1.dropna()

ds2.isnull().sum()

product_id                  0
product_category_name       0
product_id_index            0
order_id                    0
customer_id                 0
customer_unique_id          0
review_score                0
customer_unique_id_index    0
dtype: int64

In [16]:
# cannot convert a pandas datafram to pyspark if there is null value

psdf = spark.createDataFrame(ds2)
psdf.show()

+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+------------+------------------------+
|          product_id|product_category_name|product_id_index|            order_id|         customer_id|  customer_unique_id|review_score|customer_unique_id_index|
+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+------------+------------------------+
|1e9e8ef04dbcff454...|           perfumaria|               0|e17e4f88e31525f7d...|f8a3e963a310aa58b...|b1a1199364a4a7fe2...|           5|                   18226|
|3aa071139cb16b67c...|                artes|               1|5236307716393b711...|03fc97548af8f58fe...|4b86049cb99e4aa77...|           5|                   82684|
|96bd76ec8810374ed...|        esporte_lazer|               2|01f66e58769f84129...|e41819d1c95c12c9c...|f63805d9c7edb84d9...|           5|                   65693|
|cef67bcfe19066a93...|

# Create and Train 1st Model

In [17]:
#import library for ALS algorithm
from pyspark.ml.evaluation import RegressionEvaluator
 
from pyspark.ml.recommendation import ALS

(training, test) = psdf.randomSplit([0.8, 0.2])

#set up ALS algorithm
als = ALS(maxIter=5, regParam=0.01, userCol="customer_unique_id_index", itemCol="product_id_index", ratingCol="review_score",
          coldStartStrategy="drop")

model = als.fit(training)

In [18]:
# evaluate predictions

predictions=model.transform(test)

evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="review_score", 
           predictionCol="prediction")

rmse=evaluator.evaluate(predictions)

print("rmse for the model: ", rmse) # it means that the average prediction error is 2.3 above or below the review score.

rmse for the model:  2.3057677916538815


## Improving model using hyperparameter tuning and cross-validation

In [19]:
# Hyperparameter tuning and cross-validation

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()

# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="review_score", 
           predictionCol="prediction") 

print ("Num models to be tested: ", len(param_grid))

# Build cross validation using CrossValidator

cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

Num models to be tested:  16


In [181]:
# Training 16 models (TAKES FEW HOURS)

#Fit cross validator to the 'train' dataset
model = cv.fit(training)

#Extract best model from the cv model above
best_model = model.bestModel

# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)

print(RMSE) # the model has improved after runing cross validation and hyperparameter tunning

2.0786688703798344


In [182]:
print("**Best Model**")
# Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())
# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

**Best Model**
  Rank: 150
  MaxIter: 5
  RegParam: 0.15


# using the above hyperparameter to run the model again

In [20]:
from pyspark.ml.evaluation import RegressionEvaluator
 
from pyspark.ml.recommendation import ALS

(training, test) = psdf.randomSplit([0.8, 0.2])

als = ALS(maxIter=5, regParam=0.15, rank=150, userCol="customer_unique_id_index", itemCol="product_id_index", ratingCol="review_score",
          coldStartStrategy="drop")

model = als.fit(training)

In [21]:
predictions=model.transform(test)

evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="review_score", 
           predictionCol="prediction")

rmse=evaluator.evaluate(predictions)

print("rmse for the model: ", rmse) # it means that the average prediction error is 2.09 above or below the review score.

rmse for the model:  2.1204150538319357


## Generate n recommendations for all customers

In [22]:
recommendations=model.recommendForAllUsers(5)

recommendations.show()

+------------------------+--------------------+
|customer_unique_id_index|     recommendations|
+------------------------+--------------------+
|                      31|[{27539, 3.931027...|
|                      34|[{28675, 4.974535...|
|                      53|[{7345, 1.4270169...|
|                      65|[{960, 4.9408255}...|
|                      78|[{25473, 1.011703...|
|                      85|[{27765, 2.939042...|
|                     108|[{6965, 4.92919},...|
|                     133|[{32836, 4.924059...|
|                     137|[{19357, 3.913703...|
|                     148|[{30115, 4.928243...|
|                     155|[{1575, 3.9415877...|
|                     193|[{22336, 1.045303...|
|                     211|[{30304, 4.921156...|
|                     243|[{17127, 0.995686...|
|                     251|[{21415, 4.937418...|
|                     255|[{4378, 4.933041}...|
|                     296|[{2209, 4.9282427...|
|                     321|[{27602, 4.928

In [23]:
allCustomer_recommendations = recommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('customer_unique_id_index', col("rec_exp.product_id_index"), col("rec_exp.rating"))

allCustomer_recommendations.show()

+------------------------+----------------+---------+
|customer_unique_id_index|product_id_index|   rating|
+------------------------+----------------+---------+
|                      31|           27539|3.9310272|
|                      31|            8322|1.7417027|
|                      31|           32156|1.4154928|
|                      31|           23394|1.3439854|
|                      31|           20952|1.3177129|
|                      34|           28675|4.9745355|
|                      34|           15505|4.9616876|
|                      34|            3129| 4.842282|
|                      34|            8340|3.8301466|
|                      34|           16893|1.6985462|
|                      53|            7345|1.4270169|
|                      53|           14051|  1.42218|
|                      53|            4598| 1.364977|
|                      53|           18549|1.2983416|
|                      53|           22672|1.2795619|
|                      65|  

In [25]:
psdf1=psdf.select('product_id_index','product_category_name').distinct()

psdf1.show()

+----------------+---------------------+
|product_id_index|product_category_name|
+----------------+---------------------+
|              88|     moveis_decoracao|
|             126|        esporte_lazer|
|           13047|            papelaria|
|           23440|        esporte_lazer|
|            1221|           cool_stuff|
|             391|      cama_mesa_banho|
|             479| fashion_bolsas_e_...|
|             510| sinalizacao_e_seg...|
|             562|         beleza_saude|
|             712| utilidades_domest...|
|             772|        esporte_lazer|
|            1261|            papelaria|
|            1340|      cama_mesa_banho|
|            1423|         beleza_saude|
|           13905|             pet_shop|
|             293| construcao_ferram...|
|             603|           cool_stuff|
|             835|           perfumaria|
|             908|           perfumaria|
|            1234| informatica_acess...|
+----------------+---------------------+
only showing top

In [26]:
allcustomers_results=allCustomer_recommendations.join(psdf1, 'product_id_index', 'inner').sort('customer_unique_id_index', 'rating', ascending=False)

allcustomers_results.show(truncate=False)

+----------------+------------------------+----------+----------------------+
|product_id_index|customer_unique_id_index|rating    |product_category_name |
+----------------+------------------------+----------+----------------------+
|11534           |94719                   |2.9594886 |relogios_presentes    |
|9671            |94719                   |2.2668178 |cool_stuff            |
|24622           |94719                   |0.93350506|moveis_decoracao      |
|7772            |94719                   |0.8953066 |brinquedos            |
|24385           |94719                   |0.8696679 |perfumaria            |
|11534           |94718                   |3.945986  |relogios_presentes    |
|9671            |94718                   |3.0224242 |cool_stuff            |
|24622           |94718                   |1.2446734 |moveis_decoracao      |
|7772            |94718                   |1.1937417 |brinquedos            |
|24385           |94718                   |1.1595571 |perfumaria

In [ ]:
# write the results to a csv file


allcustomers_results_pd=allcustomers_results.toPandas()
allcustomers_results_pd = allcustomres_results_pd.sort('customer_unique_id_index', 'rating', ascending=False)

allcustomers_results_pd.to_csv('all_customer_top5recommendations.csv', index=False)

# Generate n recommendations for selected customers

In [43]:
# provide id of a customer or a list of customer ID

customer_ids=[0, 1, 2]

selectedCustomers=psdf.filter(col('customer_unique_id_index').isin(customer_ids)).select('customer_unique_id_index')

top5 = model.recommendForUserSubset(selectedCustomers, 5)

top5.show(truncate=False)

+------------------------+-----------------------------------------------------------------------------------------------------+
|customer_unique_id_index|recommendations                                                                                      |
+------------------------+-----------------------------------------------------------------------------------------------------+
|1                       |[{18044, 0.993787}, {14912, 0.9834209}, {25905, 0.98282266}, {29768, 0.9331304}, {18982, 0.91034955}]|
|2                       |[{18053, 0.98274815}, {998, 0.92313194}, {7461, 0.8511715}, {16218, 0.84914976}, {2742, 0.84914976}] |
|0                       |[{15672, 3.9061022}, {20357, 1.55108}, {16927, 1.5503998}, {8119, 1.4228102}, {10617, 1.4222896}]    |
+------------------------+-----------------------------------------------------------------------------------------------------+



In [45]:
customer_ids = ['b1a1199364a4a7fe27c4486ab63f550d']

selectedCustomers = psdf.filter(col('customer_unique_id').isin(customer_ids)).select('customer_unique_id_index')

top5 = model.recommendForUserSubset(selectedCustomers, 5)


top5.show(truncate=False)

+------------------------+----------------------------------------------------------------------------------------------+
|customer_unique_id_index|recommendations                                                                               |
+------------------------+----------------------------------------------------------------------------------------------+
|18226                   |[{0, 4.9282436}, {4765, 1.692111}, {11393, 1.4246727}, {24792, 1.4176062}, {23773, 1.4134758}]|
+------------------------+----------------------------------------------------------------------------------------------+



In [28]:
from pyspark.sql.functions import explode, col
 
nrecommendations = top5\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('customer_unique_id_index', col("rec_exp.product_id_index"), col("rec_exp.rating"))

nrecommendations.show()

+------------------------+----------------+----------+
|customer_unique_id_index|product_id_index|    rating|
+------------------------+----------------+----------+
|                       1|           18044|  0.993787|
|                       1|           14912| 0.9834209|
|                       1|           25905|0.98282266|
|                       1|           29768| 0.9331304|
|                       1|           18982|0.91034955|
|                       2|           18053|0.98274815|
|                       2|             998|0.92313194|
|                       2|            7461| 0.8511715|
|                       2|           16218|0.84914976|
|                       2|            2742|0.84914976|
|                       0|           15672| 3.9061022|
|                       0|           20357|   1.55108|
|                       0|           16927| 1.5503998|
|                       0|            8119| 1.4228102|
|                       0|           10617| 1.4222896|
+---------

In [29]:
products_index=psdf.select('product_id_index','product_id', 'product_category_name').distinct()

products_index.show()

+----------------+--------------------+---------------------+
|product_id_index|          product_id|product_category_name|
+----------------+--------------------+---------------------+
|              23|8b41fbc2b984a1203...|                bebes|
|             122|c5fb7fbe39f4e09e0...| sinalizacao_e_seg...|
|             340|eca26be90d3b3454d...| fashion_bolsas_e_...|
|             398|78bd107dbc2f94f45...| utilidades_domest...|
|           20422|5ac3db3f4250d6add...|     moveis_decoracao|
|           12750|9a2e821cfc5130513...|            papelaria|
|           11346|c10d842a54be20359...|        esporte_lazer|
|             981|407eeaa1c0dcd776f...|            telefonia|
|            1045|b0069f9a7b1811276...|          eletronicos|
|            1125|d5448c89ef7dc8228...|            telefonia|
|            1217|8e9ecca4227272ded...|     moveis_decoracao|
|            2791|e4aa3f3e320900916...|   ferramentas_jardim|
|           19149|4256cf575577f02e6...|     moveis_decoracao|
|       

In [30]:
customers_index=psdf.select('customer_unique_id', 'customer_unique_id_index').distinct()

customers_index.show()

+--------------------+------------------------+
|  customer_unique_id|customer_unique_id_index|
+--------------------+------------------------+
|dbe1f72423b35ecd3...|                    8959|
|79444cb5bb16964ee...|                    8844|
|50d38ebfa2476b95a...|                   65626|
|e257ea35efcc4d9b3...|                   66054|
|0d589ee71b3f046e5...|                    8634|
|2c14869f96fcf1d76...|                   65063|
|6e279371383dfb449...|                   64911|
|6e9a6867883241de5...|                     746|
|5b9603bf6b5ba1130...|                   65391|
|00e336519abc0012c...|                   65284|
|5ebf36a0459a0403b...|                   67131|
|91b628eaecfe80787...|                   67097|
|50208f415cd5914be...|                   67083|
|3996afa592a519717...|                   66357|
|8cce325d75f2cc112...|                    7179|
|4ed82d2c4fb7be1ab...|                   66681|
|6836229709adeb622...|                   62931|
|24009d47bd49ded6c...|                  

In [32]:
result1=nrecommendations.join(customers_index, 'customer_unique_id_index', 'inner')
result2=result1.join(products_index, 'product_id_index', 'inner').sort('customer_unique_id_index', 'rating', ascending=False)

result2.show(truncate=False)

+----------------+------------------------+----------+--------------------------------+--------------------------------+---------------------+
|product_id_index|customer_unique_id_index|rating    |customer_unique_id              |product_id                      |product_category_name|
+----------------+------------------------+----------+--------------------------------+--------------------------------+---------------------+
|18053           |2                       |0.98274815|4546caea018ad8c692964e3382debd19|f6e3aeb40c00aec24ade8333c2ac1b20|consoles_games       |
|998             |2                       |0.92313194|4546caea018ad8c692964e3382debd19|35e26f65c3f380cfe6257d771fb067d2|esporte_lazer        |
|7461            |2                       |0.8511715 |4546caea018ad8c692964e3382debd19|4066f66e5bd7a5371448d70eeb24b9dc|esporte_lazer        |
|2742            |2                       |0.84914976|4546caea018ad8c692964e3382debd19|270516a3f41dc035aa87d220228f844c|beleza_saude         |

# Organizing and Writing All Results to File 

In [33]:
# write the results to a csv file
allcustomers_results1 =allcustomers_results.join(customers_index, 'customer_unique_id_index', 'inner')
allcustomers_results2=allcustomers_results1.join(products_index, 'product_id_index', 'inner')

allcustomers_results2.show()
allcustomers_results2 = allcustomers_results2.sort('customer_unique_id_index', 'rating', ascending=False)

allcustomers_results_pd=allcustomers_results2.toPandas()

+----------------+------------------------+----------+---------------------+--------------------+--------------------+---------------------+
|product_id_index|customer_unique_id_index|    rating|product_category_name|  customer_unique_id|          product_id|product_category_name|
+----------------+------------------------+----------+---------------------+--------------------+--------------------+---------------------+
|           28623|                   40132| 1.4922339|     moveis_decoracao|2b12a51b531dea88f...|bbbfa3b8e4c5b134e...|     moveis_decoracao|
|           28623|                   51418| 1.4443693|     moveis_decoracao|70dad727e56e0f608...|bbbfa3b8e4c5b134e...|     moveis_decoracao|
|           30892|                   11745| 3.9342055|     moveis_decoracao|c5157b7836a33d547...|c7fd13b5e515bffda...|     moveis_decoracao|
|            4762|                   36103| 1.1305987| informatica_acess...|8837a3d0a8c6ca2e5...|5450ae624c196f64c...| informatica_acess...|
|            

In [34]:
columns_name=['product_id_index', 'customer_unique_id_index', 'rating',
       'product_category_name', 'customer_unique_id', 'product_id', 'product_category_name_02']

allcustomers_results_pd.columns=columns_name

In [35]:
new_df=allcustomers_results_pd.drop(['product_category_name_02'], axis=1)

new_df.head()


,product_id_index,customer_unique_id_index,rating,product_category_name,customer_unique_id,product_id
0,11534,94719,2.959489,relogios_presentes,96ce6d85f1fd6f113fa1298a27047f2a,7e6c4a0bf900e259f50ba63331fd2785
1,9671,94719,2.266818,cool_stuff,96ce6d85f1fd6f113fa1298a27047f2a,5272e64de887c95d7a1c9c64886b90f5
2,24622,94719,0.933505,moveis_decoracao,96ce6d85f1fd6f113fa1298a27047f2a,4fcac8e09620cd87a5e5283e6748da67
3,7772,94719,0.895307,brinquedos,96ce6d85f1fd6f113fa1298a27047f2a,6b3c88110203820df52c762cd2d2e80f
4,24385,94719,0.869668,perfumaria,96ce6d85f1fd6f113fa1298a27047f2a,a01c3a8e3ccddf44069c082b1590f19a


In [36]:
new_df.to_csv('all_customer_top5recommendations.csv', index=False)